In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "EMBEDDIA/crosloengual-bert"

In [3]:
from datasets import load_dataset

dataset = load_dataset("lenatr99/Slovene_SuperGLUE_CB")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
id2label = {0: "entailment", 1: "neutral", 2: "contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
model.config.use_cache = False

C:\Users\anzeo\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/crosloengual-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
CONTEXT_COL = "premise"
HYPOTHESIS_COL = "hypothesis"


def preprocess_function(examples):
    """
    The preprocessing function prepares examples for processing by the model.
    It concatenates premise and hypothesis for each example to form a single input string.
    """
    inputs = [f"{premise} {hypothesis}" for premise, hypothesis in zip(examples[CONTEXT_COL], examples[HYPOTHESIS_COL])]
    tokenized_examples = tokenizer(inputs, truncation=True)
    if "label" in examples:
        tokenized_examples["labels"] = [label2id[label] for label in examples["label"]]
    return tokenized_examples

In [7]:
tokenized_dataset = dataset.map(preprocess_function,
                                remove_columns=['idx', 'premise', 'hypothesis', 'label'], batched=True)

In [8]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 110
    })
    eval: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 22
    })
})

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
import numpy as np

from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [5]:
from transformers import set_seed

set_seed(42)

In [2]:
from transformers import TrainingArguments

new_model_name = "fine_tuned_cb_croslo"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [14]:
trainer.train()

  0%|          | 0/400 [00:00<?, ?it/s]

{'loss': 0.7054, 'grad_norm': 4.0136637687683105, 'learning_rate': 1.7500000000000002e-05, 'epoch': 3.57}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.4024447202682495, 'eval_accuracy': 0.3181818181818182, 'eval_f1': 0.1536050156739812, 'eval_runtime': 0.3936, 'eval_samples_per_second': 55.892, 'eval_steps_per_second': 7.622, 'epoch': 3.57}
{'loss': 0.3117, 'grad_norm': 0.8568326830863953, 'learning_rate': 1.5000000000000002e-05, 'epoch': 7.14}


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.2962737083435059, 'eval_accuracy': 0.6818181818181818, 'eval_f1': 0.6389986824769434, 'eval_runtime': 0.468, 'eval_samples_per_second': 47.007, 'eval_steps_per_second': 6.41, 'epoch': 25.0}
{'loss': 0.0015, 'grad_norm': 0.034898024052381516, 'learning_rate': 0.0, 'epoch': 28.57}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.3248982429504395, 'eval_accuracy': 0.6818181818181818, 'eval_f1': 0.6389986824769434, 'eval_runtime': 0.466, 'eval_samples_per_second': 47.207, 'eval_steps_per_second': 6.437, 'epoch': 28.57}
{'train_runtime': 160.1036, 'train_samples_per_second': 19.987, 'train_steps_per_second': 2.498, 'train_loss': 0.1321098879724741, 'epoch': 28.57}


TrainOutput(global_step=400, training_loss=0.1321098879724741, metrics={'train_runtime': 160.1036, 'train_samples_per_second': 19.987, 'train_steps_per_second': 2.498, 'total_flos': 187764312726144.0, 'train_loss': 0.1321098879724741, 'epoch': 28.571428571428573})

In [15]:
trainer.evaluate()

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.3248982429504395,
 'eval_accuracy': 0.6818181818181818,
 'eval_f1': 0.6389986824769434,
 'eval_runtime': 0.4721,
 'eval_samples_per_second': 46.604,
 'eval_steps_per_second': 6.355,
 'epoch': 28.571428571428573}

In [16]:
trainer.push_to_hub(new_model_name)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/497M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lenatr99/fine_tuned_cb_croslo/commit/b4365bec3f84cb10d1fc5dd148d22ec39104d7c7', commit_message='fine_tuned_cb_croslo', commit_description='', oid='b4365bec3f84cb10d1fc5dd148d22ec39104d7c7', pr_url=None, pr_revision=None, pr_num=None)

In [9]:
# Example
hypothesis = "Valence je pomagal"
premise = "Valence praznoglavi, Valence krepostni kreten. Zakaj si ga tip raje ni zataknil v ustrezen del lastne titanske anatomije? Je morda mislil, da mi pomaga?"

In [10]:
import torch

# We need to set the seed, otherwise some weights of the model are initialized differently every time, and consequently the result can be different each time as well
set_seed(42)

adapter_name = "lenatr99/" + new_model_name

tokenizer = AutoTokenizer.from_pretrained(adapter_name)
model = AutoModelForSequenceClassification.from_pretrained(
    adapter_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)

config.json:   0%|          | 0.00/874 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/497M [00:00<?, ?B/s]

In [11]:
inputs = tokenizer(f"{premise} {hypothesis}", return_tensors="pt")
label = torch.tensor([1]).unsqueeze(0)  # Batch size 1

outputs = model(**inputs, labels=label)
logits = outputs.logits

In [12]:
# Print prediction
logits.argmax().item()

0